In [8]:
"""
 author:jjk
 datetime:2019/11/30
 coding:utf-8
 project name:Pycharm_workstation
 Program function:  30万条新闻文本数据清洗
 
"""
#from stopword import readFile,seg_doc
import os
import time
import jieba
import re

In [9]:
# 加载文件夹
class loadFolders(object):#迭代器
    def __init__(self,par_path):
        self.par_path = par_path
    def __iter__(self):
        for file in os.listdir(self.par_path):
            file_abspath = os.path.join(self.par_path,file)
            if os.path.isdir(file_abspath): # if file is a folder
                yield file_abspath
                

class loadFiles(object):
    def __init__(self,par_path):
        self.par_path = par_path
    def __iter__(self):
        folders = loadFolders(self.par_path) # 加载文件夹
        for folder in folders:
            catg = folder.split(os.sep[-1])
            for file in os.listdir(folder): # 二级目录(子目录),如果文件名在(os.listdir())方法用于返回指定的文件夹包含的文件或者文件夹的名字的列表
                file_path = os.path.join(folder,file) # 文件的路径结合
                # 文件的具体操作
                if os.path.isfile(file_path):
                    this_file = open(file_path,'rb') # 二进制读取更快
                    content = this_file.read().decode('utf8') # 读取内容
                    yield catg,content
                    this_file.close() # 关闭文件流

In [12]:
# 读取文本信息
def readFile(path):
    str_doc = ""
    with open(path,'r',encoding='utf-8') as f:
        str_doc = f.read()
    return str_doc

# 正则对字符串清洗
def textParse(str_doc):
    # 去掉字符
    str_doc = re.sub('\u3000','',str_doc)
    return str_doc


# 创建停用词列表
def get_stop_words(path=r'../dataSet/StopWord/NLPIR_stopwords.txt'):
    file = open(path,'r',encoding='utf-8').read().split('\n')
    return set(file)


# 去掉一些停用词和数字
def rm_tokens(words,stwlist):
    words_list = list(words)
    stop_words = stwlist
    for i in range(words_list.__len__())[::-1]:
        if words_list[i] in stop_words: # 去除停用词
            words_list.pop(i)
        elif words_list[i].isdigit(): # 去除数字
            words_list.pop(i)
        elif len(words_list[i]) == 1:  # 去除单个字符
            words_list.pop(i)
        elif words_list[i] == " ":  # 去除空字符
            words_list.pop(i)
    return words_list


# 利用jieba对文本进行分词，返回切词后的list
def seg_doc(str_doc):
    sent_list = str_doc.split('\n')
    # 1、正则处理，去掉一些字符，例如\u3000
    sent_list = map(textParse,sent_list)
    # 2、获取停用词
    stwlist = get_stop_words()
    # 3、分词并去除停用词
    word_2dlist = [rm_tokens(jieba.cut(part,cut_all=False),stwlist) for part in sent_list]
    # 4、合并列表
    word_list = sum(word_2dlist,[])
    return word_list


    
    



In [16]:
# 主函数
if __name__=='__main__':
    start = time.time()
    # 文件读取操作
    filepath = os.path.abspath(r'../dataSet/CSCMNews')# 读取所有文件
    files = loadFiles(filepath)
    n = 3  # n 表示抽样率， n抽1
    for i, msg in enumerate(files):
        # 内容，类别
        if i % n == 0:
            catg = msg[0][-1]# 类别， 或者catg = msg[0]
            content = msg[1] # 内容
            content = seg_doc(content)  # 分词的一个处理
            if int(i/n) % 1000 == 0: # 每隔3000条打印
                print('{t} *** {i} \t docs has been dealed'
                      .format(i=i, t=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())),'\n',catg,':\t',content[:30]) # 只取每条文档前30个词

    end = time.time()
    print('total spent times:%.2f' % (end-start)+ ' s')
    

2019-12-01 13:16:12 *** 0 	 docs has been dealed 
 体育 :	 ['解读', '范帅', '惨淡', '球员', '生涯', '藉藉无名', '太极', '巧解', '内讧', '顽疾', '巴斯滕', '杰卡尔', '赫赫的', '球员', '经历', '缺少', '希丁克', '艾德沃', '卡特', '传奇', '执教', '履历', '马尔维', '这位', '荷兰', '之外', '藉藉无名', '教练', '南非', '世界杯']
2019-12-01 13:16:24 *** 3000 	 docs has been dealed 
 体育 :	 ['现场', '中国女排', '次局', '被动', '徐云丽', '登场', '新浪', '体育讯', '第二局', '中国女排', '先得', '两分', '多米尼加', '顽强', '比分', '扳成', '薛明', '快攻', '多米尼加', '号位', '命中', '达成', '多米尼加', '被判', '触网', '中国队', '一分', '反超', '多米尼加', '一记']
2019-12-01 13:16:35 *** 6000 	 docs has been dealed 
 体育 :	 ['男子', '冰壶', '中国队', '大胜', '丹麦', '赢得', '冬奥会', '首场', '胜利', '新浪', '体育讯', '北京', '时间', '冬奥会', '男子', '冰壶', '单循环赛', '第三轮', '一场', '比赛', '中国男队', '大胜', '丹麦队', '赢下', '中国', '冰壶', '奥运', '赛场', '第一场', '比赛']
2019-12-01 13:16:46 *** 9000 	 docs has been dealed 
 体育 :	 ['申雪', '赵宏博', '第三度', '最佳', '组合', '渴望', '教练', '新浪', '体育讯', '北京', '时间', '日晚', '体坛', '风云人物', '年度', '评选', '颁奖', '盛典', '国家', '会议', '中心', '隆重', '颁出', '奖项', '年度', '最佳', '组合', '奥运会', '花样滑冰',